## Setup

In [2]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
# matplotlib.backend
# matplotlib.validate_backend("GTK3Ag")

In [3]:
import numpy as np
from tqdm.auto import tqdm as progressbar
import matplotlib.pyplot as plt
import pickle
import matplotlib.animation as animation
import cv2
import cv2.aruco as aruco
import common as c
from os.path import join
from calib import calibrate_charuco_local, load_board, load_coefficients, save_coefficients

In [4]:
data_path="/home/freitas/TCC/ender-laser-scanner/pics/v1_raspi/"
mtx, dist = load_coefficients("/home/freitas/TCC/ender-laser-scanner/calib_data/calibration_charuco.yml")

In [5]:
import json

board, aruco_dict = load_board("board.pkl")

print(board.getChessboardSize())
print(board.getMarkerLength())
print(board.getSquareLength())

(16, 16)
9.375
12.5


In [6]:
def load(path):
    info = {}
    data = []
    with open(join(path,"points.json")) as f:
        info = json.loads(f.read())

    for fig, point in info.items():
        image =  cv2.cvtColor(cv2.imread(join(path, fig)), cv2.COLOR_BGR2RGB)
        image = cv2.undistort(image, mtx, dist, None, mtx)
        data.append( (point, image) )

    return data


arucoParams = aruco.DetectorParameters_create()

def find_charuco_pose(image, mtx, dist):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers( gray, aruco_dict, cameraMatrix=mtx, parameters=arucoParams, distCoeff=dist)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco( markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=mtx, distCoeffs=dist)

    if(resp > 6):
        retval, rvec, tvec	=	cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, mtx, dist, None,None)
    else:
        return None, None
    if not retval:
        print("erro")
    return rvec, tvec

In [7]:
data = load(data_path+"auto_exp")
coords = []

for point, image in data:
    rvec, tvec = find_charuco_pose(image, mtx, None)
    if rvec is None or tvec is None:
        print("OOPS")
    coords.append((rvec, tvec))

data = load(data_path+"low_exp")

new_coords = []
new_data = []
for (point, image), (rvec, tvec) in zip(data, coords):
    if rvec is None or tvec is None:
        print("OOPS")
        continue
    new_data.append((point,image))
    new_coords.append((rvec, tvec))

data = new_data
coords = new_coords

In [25]:
import common as c
c.animate((g[1] for g in data))

## Funcs

In [8]:
def find_charuco_corners(image, mtx, dist):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers( gray, aruco_dict, cameraMatrix=mtx, distCoeff=dist, parameters=arucoParams)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco( markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=mtx, distCoeffs=dist)
    return charuco_corners


def mask_rgb(img, mask):
	ret = np.zeros_like(img)
	for i in range(ret.shape[-1]):
		ret[:,:,i] = img[:,:,i] * mask
	return ret

def get_charuco_rect(board):
	delim = np.max(board.chessboardCorners, axis=0)
	b =board.getSquareLength() 
	corner_points = [
		[b, b, 0],
		[delim[0], b, 0],
		delim,
		[b, delim[1], 0],
	]
	return np.array(corner_points)

def get_charuco_mask(image, board, rvec, tvec, mtx, dist):
	if image.ndim==2:
		mask = np.zeros_like(image)
	elif image.ndim==3:
		mask = np.zeros(image.shape[:2])

	rect = get_charuco_rect(board)
	points, _ = cv2.projectPoints(rect, rvec, tvec, mtx, dist)
	proj_points_round = np.round(points, 0).astype(np.int32)
	cv2.fillPoly(mask, [proj_points_round], True, 255 )
	return mask

def crop_charuco_board(image, board, rvec, tvec, mtx, dist):
	mask = get_charuco_mask(image, board, rvec, tvec, mtx, dist)
	if image.ndim == 3:
		return mask_rgb(image, mask)
	elif image.ndim==2:
		return mask*image

def chess_plane(rvec, tvec):
	# Find p
	n_chessboard = np.array([0, 0, 1])
	M,_ = cv2.Rodrigues(rvec)
	n3 = M@n_chessboard
	p = -1 * n3@tvec
	n = np.hstack((n3,p))
	return n

# Laser

## Inspecionar Laser

In [9]:
def find_board_and_laser(image, rvec, tvec, mtx, dist):
    """Crop out the image to contain only charuco board, and find the laser points in this image
    @param image original image
    @param rvec rotation vec of the board
    @param tvec translation vec of the board
    @returns: (masked, centroids), tuple containing the masked image with only the board, and vector of laser points """
    masked = crop_charuco_board(image, board, rvec, tvec, mtx, dist)
    centroids = c.column_centroids(c.red_contrast(masked), mask=c.red_contrast(masked)>50)
    try:
        p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T
    except ValueError:
        p_centroids = np.array([[],[]])
    return masked, p_centroids

### Plot de todas as linhas

In [28]:
n=0
for (point, image), (rvec, tvec) in zip(data, coords):
    masked, p_centroids = find_board_and_laser(image, rvec, tvec, mtx, None)
    p_centroids = p_centroids[:,50:-50]
    plt.plot(p_centroids[0], p_centroids[1], label=n)
    n+=1
plt.legend() 

### Inspecionar linha específica

### Animar Laser

In [30]:
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None

def init():
    global imgplot, pointsplot
    imgplot = plt.imshow(data[0][1], animated=True)
    pointsplot, = plt.plot(range(data[0][1].shape[0]), "r", animated=True)
    return pointsplot, imgplot

def laser_points_img_gen(i):
    global imgplot, pointsplot
    # Find board and laser
    point, image = data[i]
    masked, centroids = find_board_and_laser(image, *coords[i], mtx, None)
    print(masked.shape)

    # Update anim
    imgplot.set_data(masked)
    centroids = centroids[:,50:-50]
    pointsplot.set_data(centroids[0], centroids[1])
    return pointsplot, imgplot

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=len(data), interval=500, blit=True)

(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3280, 3)
(2464, 3

## Animar pontos

In [34]:
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None
chessplot = None
chessplot_reproj = None


def init():
    global imgplot, pointsplot, chessplot, chessplot_reproj
    imgplot = plt.imshow(data[0][1], animated=True)
    pointsplot, = plt.plot(range(data[0][1].shape[0]), "r", animated=True)
    chessplot, = plt.plot(range(data[0][1].shape[0]), "b.", animated=True)
    chessplot_reproj, = plt.plot(range(data[0][1].shape[0]), "g.", animated=True)
    return pointsplot, imgplot, chessplot, chessplot_reproj

def laser_points_img_gen(i):
    global imgplot, pointsplot, chessplot, chessplot_reproj
    # Find board and laser
    point, image  = data[i]
    masked, centroids = find_board_and_laser(image, *coords[i], mtx, None)
    charuco_corners = find_charuco_corners(image, mtx, None)
    rvec, tvec = find_charuco_pose(image, mtx, None)
    proj_points, jacobian = cv2.projectPoints(board.chessboardCorners, rvec, tvec, mtx, None)

    # Update anim
    imgplot.set_data(image)
    pointsplot.set_data(centroids[0], centroids[1])
    chessplot.set_data(charuco_corners[..., 0], charuco_corners[..., 1])
    chessplot_reproj.set_data(proj_points[..., 0], proj_points[..., 1])
    return pointsplot, imgplot, chessplot, chessplot_reproj

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=200, interval=500, blit=True)

Traceback (most recent call last):
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/backend_bases.py", line 1216, in _on_timer
    ret = func(*args, **kwargs)
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/animation.py", line 1477, in _step
    still_going = super()._step(*args)
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/animation.py", line 1189, in _step
    self._draw_next_frame(framedata, self._blit)
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/animation.py", line 1208, in _draw_next_frame
    self._draw_frame(framedata)
  File "/home/freitas/TCC/ender-laser-scanner/tccenv/lib/python3.9/site-packages/matplotlib/animation.py", line 1776, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "/tmp/ipykernel_8887/154839419.py", line 23, in laser_points_img_gen
    charuco_corn

In [15]:
i = 19
(point, image), rvec, tvec = images[i], rvecs[i], tvecs[i]
masked, p_centroids = find_board_and_laser(image, rvec, tvec)
charuco_corners=find_charuco_corners(image)

proj_points, jacobian = cv2.projectPoints(board.chessboardCorners, rvec, tvec, mtx, dist)

plt.imshow(masked)
plt.plot(p_centroids[0],p_centroids[1])
plt.plot(charuco_corners[..., 0],charuco_corners[..., 1], "b.")
plt.plot(proj_points[..., 0],proj_points[..., 1], "g.")

NameError: name 'images' is not defined

## Plotar Eixos coords

In [10]:
import matplotlib.pyplot as plt
import importlib
import common as co
importlib.reload(co)
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

fig, ax = co.plot3d()

zs = []
t0 = None
points = pd.DataFrame(columns=["c1","c2","c3","space","p1","p2","p3"])
for (point, image), (rvec, tvec) in zip(data, coords):

    inv = np.linalg.inv(co.vec2M(rvec, tvec))
    co.plot_axes(ax, inv)

    tvec = inv[:3,3]
    if t0 is None:
        t0 = tvec
    cam = (tvec.T - t0.T).flatten()
    p = point
    row = dict(c1=cam[0],c2=cam[1],c3=cam[2], p1=p[0],p2=p[1],p3=p[2])
    points = points.append(row, ignore_index=True)

    zs.append(tvec[-1])

# Chessboard
ax.scatter(board.chessboardCorners[:,0], board.chessboardCorners[:,1], board.chessboardCorners[:,2], color="k", label="board")
co.set_axes_equal(ax)
plt.legend()

In [36]:
inv

array([[ 9.98715816e+00, -2.52820754e-02,  5.05996794e-01,
         7.44844357e+01],
       [-2.78614496e-01, -8.61572401e+00,  5.06869546e+00,
         2.35200322e+01],
       [ 4.23138159e-01, -5.07628413e+00, -8.60536423e+00,
         1.54264969e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [37]:
points["p1"] -= 70.0
points["p2"] -= 50.0
points["p3"] -= 100.0

In [38]:
points["space"] = ""
points.round(1).head(n=20)

,c1,c2,c3,space,p1,p2,p3
0,0.0,0.0,0.0,,0.0,0.0,0.0
1,53.8,0.2,-1.0,,55.0,0.0,0.0
2,107.5,0.7,-0.5,,110.0,0.0,0.0
3,107.6,45.7,-0.4,,110.0,45.0,0.0
4,54.0,45.3,-0.8,,55.0,45.0,0.0
5,0.1,45.1,0.2,,0.0,45.0,0.0
6,0.4,90.2,0.5,,0.0,90.0,0.0
7,54.2,90.3,-0.7,,55.0,90.0,0.0
8,107.8,90.5,-0.5,,110.0,90.0,0.0
9,0.5,0.2,19.9,,0.0,0.0,20.0


In [41]:
plt.scatter(points["c1"], points["c2"], label="camera")
plt.scatter(points["p1"], points["p2"], label="point")
plt.legend()
# plt.scatter(points["c1"], points["c2"])

In [ ]:
import pandas as pd

print(pd.Series(zs).describe())
print(max(zs) - min(zs))

## Calibrar

In [11]:
def ray_plane_intersect_vectorized(img_points, plane, cam_mtx):
	"""
	Calculate 3d-intersection between image rays defined by `img_ponts` and `cam_mtx`
	and a plane defined by the 4-vec `plane`
	@param img_points np.array with shape (3, N), N= number of points, homegeneous coords
	@param plane np.array with shape = (4,)
	@param cam_mtx np.array with shape = (3,3), intrinsic camera parameters
	@returns np.array with shape (4, N) with 3d points
	"""
	assert plane.shape == (4,)
	assert len(img_points.shape) == 2
	assert img_points.shape[0] == 3

	rays = np.linalg.inv(cam_mtx)@img_points
	p = plane[-1]
	n = plane[:3]
	out = (-p/(n@rays)) * rays
	out = np.vstack((out, np.ones((1, out.shape[-1])))) # homogeneous coords
	return out

### Achar todos os pontos 3D

In [12]:
from collections import defaultdict
Xlist = []

zcounter = defaultdict(int)
for (point, image), (rvec, tvec) in zip(data, coords):
	# count z 
	zcounter[point[-1]] += 1

	masked, p_centroids = find_board_and_laser(image, rvec, tvec, mtx, None)
	n = chess_plane(rvec, tvec)
	try:
		points3d = ray_plane_intersect_vectorized(p_centroids, n, mtx)
	except Exception as e:
		print(e)
		continue
	Xlist.append(points3d)
print(zcounter)

defaultdict(<class 'int'>, {100.0: 9, 120.0: 10, 140.0: 10, 160.0: 10, 180.0: 10, 175: 1, 170: 1, 165: 1, 155: 1, 150: 1, 145: 1, 135: 1, 130: 1, 125: 1, 115: 1, 110: 1, 105: 1})


In [13]:
X = np.hstack(Xlist)

### Mostrar pontos 3D

In [14]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[0],X[1], X[2], color="r")

### Fit num plano

In [15]:
u, s, vt = np.linalg.svd(X.T, full_matrices=False)
v = vt.T
n = v[:,-1]
print(n)

[ 2.61353362e-04  9.75435153e-03 -5.73757242e-03  9.99935930e-01]


### Plot Plano e pontos

In [16]:
x0, xf = np.min(X[0]), np.max(X[0])
y0, yf = np.min(X[1]), np.max(X[1])
xplot = np.linspace(x0,xf)
yplot = np.linspace(y0,yf)
xplot, yplot = np.meshgrid(xplot, yplot)
a,b,c,d = n
zplot = -(a*xplot + b*yplot + d)/c

# Resultado

## Plot

In [17]:
print(np.mean((n/np.linalg.norm(n[:3]))@X))
print(np.mean(n@X))

0.0006404746682901288
7.249975683288139e-06


In [18]:
np.savetxt("tex/mtx.txt", mtx, delimiter=' & ', fmt='%.2f', newline=' \\\\\n')
np.savetxt("tex/n.txt", n/np.linalg.norm(n[:3]) , delimiter=' & ', fmt='%.2e', newline=' \\\\\n')
np.savetxt("tex/dist.txt", dist, delimiter=' & ', fmt='%.2f', newline=' \\\\\n')

In [19]:
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.scatter(X[0],X[1], X[2], color="r")
ax.plot_surface(xplot, yplot, zplot, color="b", alpha=0.5)

# ax_proj = fig.add_subplot(122, projection='3d')
# ax_proj.plot(X[0],X[1])

## Params

In [20]:
print("=== Intrinsic Camera Params ===")
print("===== Camera Matrix =====")
print(mtx)
print("===== Lens Distortion Params =====")
# print(dist.T)
print("=== Laser Params ===")
print("==== Laser Plane 4-vec ====")
print(n)
print("==== Laser Plane Equation ====")
print(f"{1e3*a:.2f}x + {1e3*b:.2f}y + {1e3*c:.2f}z + {1e3*d:.2f} = 0")

=== Intrinsic Camera Params ===
===== Camera Matrix =====
[[2.59128920e+03 0.00000000e+00 1.62581996e+03]
 [0.00000000e+00 2.59961593e+03 1.31712834e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
===== Lens Distortion Params =====
=== Laser Params ===
==== Laser Plane 4-vec ====
[ 2.61353362e-04  9.75435153e-03 -5.73757242e-03  9.99935930e-01]
==== Laser Plane Equation ====
0.26x + 9.75y + -5.74z + 999.94 = 0


## Save

In [21]:
import pickle 

calib = dict(
    mtx = mtx,
    dist= dist,
    n=n
)
with open("/home/freitas/TCC/ender-laser-scanner/calib_data/calib.pkl", "wb") as f:
    pickle.dump(calib, f)